In [1]:
from deepface import DeepFace
#liveness
import tensorflow as tf


In [2]:
import cv2, numpy, datetime, pygame, keyboard
import xlwings as xw

pygame 2.5.2 (SDL 2.28.3, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
model = 'liveness.model'
model = tf.keras.models.load_model(model)

In [4]:
#xlwings
#initialzing workbook and worksheet
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)


In [5]:
#xlwings

worksheet.range('A1').value = 'EMPLOYEE'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

employees=[]

In [6]:
e=2

i=0

entered = False
al_entered = False

In [ ]:
import os
import numpy as np  # Add this line for NumPy

cap = cv2.VideoCapture(0)

pygame.init()

pg_txt = (39, 112, 50)
pg_bg = (154, 179, 157)

font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font("freesansbold.ttf", 32)

screen = pygame.display.set_mode((960, 540))

background = pygame.image.load('background_image.jpg')
background = pygame.transform.rotozoom(background, 0, .8)
screen.blit(background, (0, 0))

# day today
t0 = datetime.date.today()
t0 = datetime.date

running = True

# Inside your while loop...
while running:
    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year

    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"
    screen.blit(background, (0, 0))

    # check
    if day != t0:
        t0 = day

        # Check if the sheet already exists
        existing_sheet = None
        for sheet in workbook.sheets:
            if sheet.name == date:
                existing_sheet = sheet
                break

        if existing_sheet:
            # Handle the case where the sheet already exists
            print(f"Sheet named '{date}' already present in workbook")
            # You can choose to skip adding or handle it differently
        else:
            # Add the sheet if it doesn't exist
            worksheet = workbook.sheets.add(date)
            worksheet.range('A1').value = 'EMPLOYEE'
            worksheet.range('B1').value = 'DATE'
            worksheet.range('C1').value = 'TIME'
            employees = []
            e = 2
            i = 0
            entered = False
            al_entered = False

    state, frame = cap.read()

    if state != True:
        break

    res = DeepFace.find(frame, db_path='./Database/', enforce_detection=False, model_name='VGG-Face')

    if len(res) > 0 and len(res[0]['identity']) > 0:
        # Extracting folder name from the path
        folder_name = os.path.basename(os.path.dirname(res[0]['identity'][0]))

        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        xmax = int(xmin + w)
        ymax = int(ymin + h)

        # liveness

        img = frame[ymin:ymax, xmin:xmax]
        img = cv2.resize(img, (32, 32))
        img = img.astype('float') / 255.0
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img, axis=0)

        liveness = model.predict(img)

        liveness = liveness[0].argmax()

        if liveness == 1:
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1)
            cv2.rectangle(frame, (xmin, ymin - 25), (xmax, ymin), (255, 255, 255), -1)
            cv2.putText(frame, folder_name, (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0),2,cv2.LINE_AA)

            # Extracting the name from the folder_name
            name = os.path.basename(folder_name)

            n_txt = pygame_font.render(folder_name, True, pg_txt, pg_bg)  # Corrected line
            n_txtrect = n_txt.get_rect()
            n_txtrect.center = (960 - 200, 540 // 2)

            d_txt = pygame_font.render(date, True, pg_txt, pg_bg)
            d_txtrect = d_txt.get_rect()
            d_txtrect.center = (960 - 200, (540 // 2) + 40)

            t_txt = pygame_font.render(time, True, pg_txt, pg_bg)
            t_txtrect = t_txt.get_rect()
            t_txtrect.center = (960 - 200, (540 // 2) + 80)

            screen.blit(n_txt, n_txtrect)
            screen.blit(d_txt, d_txtrect)
            screen.blit(t_txt, t_txtrect)

            if keyboard.is_pressed('enter'):
                if folder_name not in employees:
                    worksheet.range(f'A{e}').value = folder_name
                    worksheet.range(f'B{e}').value = date
                    worksheet.range(f'C{e}').value = time
                    employees.append(name)
                    e += 1
                    entered = True
            else: 
                al_entered = True

    if i < 30 and entered == True:
        e_txt = pygame_font.render('entered', True, pg_txt, pg_bg)
        e_txtrect = t_txt.get_rect()
        e_txtrect.center = (960 - 200, (540 // 2) + 120)  
        screen.blit(e_txt, e_txtrect)
        i += 1

    elif i < 30 and al_entered == True:
        ae_txt = pygame_font.render('already entered', True, pg_txt, pg_bg)
        ae_txtrect = t_txt.get_rect()
        ae_txtrect.center = (960 - 200, (540 // 2) + 120)  
        screen.blit(ae_txt, ae_txtrect)  # Use ae_txtrect instead of e_txtrect
        i += 1

    else:
        entered = False
        al_entered = False
        i = 0
                    
        image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1], 'BGR')
        image = pygame.transform.rotozoom(image, 0, .8)

        # round corners--
        size = image.get_size()
        rect_img = pygame.Surface(size, pygame.SRCALPHA)
        pygame.draw.rect(rect_img, (255, 255, 255), (0, 0, *size), border_radius=10)

        image = image.copy().convert_alpha()
        image.blit(rect_img, (0, 0), None, pygame.BLEND_RGB_MIN)  # Use pygame.BLEND_RGB_MIN
        # round corners--

        screen.blit(image, (20, 70))

        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running == False

cap.release()
pygame.quit()
# cv2.destroyAllWindows()


24-05-15 10:34:20 - Searching [[[156 145 123]
  [153 144 122]
  [152 144 122]
  ...
  [124 147 147]
  [126 148 148]
  [124 147 147]]

 [[154 141 119]
  [153 142 121]
  [151 143 121]
  ...
  [124 146 146]
  [123 146 146]
  [123 146 146]]

 [[150 139 117]
  [150 140 118]
  [149 140 118]
  ...
  [122 145 145]
  [121 143 146]
  [121 143 146]]

 ...

 [[126 125 102]
  [126 126 103]
  [125 126 103]
  ...
  [106 141 147]
  [106 140 149]
  [106 140 149]]

 [[127 126 103]
  [129 126 103]
  [131 127 104]
  ...
  [104 141 147]
  [104 141 147]
  [104 141 147]]

 [[127 126 103]
  [129 126 103]
  [130 126 103]
  ...
  [107 140 150]
  [106 140 149]
  [106 141 149]]] in 109 length datastore
24-05-15 10:34:23 - find function duration 3.3162219524383545 seconds
1/1 [==============================] - 0s 283ms/step
24-05-15 10:34:24 - Searching [[[170 144 121]
  [168 144 121]
  [164 142 119]
  ...
  [136 146 146]
  [137 146 146]
  [136 145 144]]

 [[163 141 118]
  [164 141 118]
  [166 142 119]
  ...
  [13

24-05-15 10:34:29 - find function duration 0.45929741859436035 seconds
1/1 [==============================] - 0s 23ms/step
24-05-15 10:34:29 - Searching [[[177 154 133]
  [177 154 133]
  [177 154 133]
  ...
  [138 152 158]
  [141 153 159]
  [141 153 159]]

 [[174 152 131]
  [175 152 131]
  [174 152 131]
  ...
  [138 153 156]
  [143 154 157]
  [142 152 156]]

 [[172 149 129]
  [170 150 129]
  [169 151 130]
  ...
  [137 153 158]
  [141 151 157]
  [140 150 156]]

 ...

 [[147 131 112]
  [145 131 112]
  [143 131 112]
  ...
  [122 147 159]
  [121 146 157]
  [124 148 160]]

 [[147 131 112]
  [146 131 112]
  [145 131 112]
  ...
  [120 147 159]
  [122 149 160]
  [122 149 160]]

 [[148 132 113]
  [148 132 114]
  [148 132 115]
  ...
  [124 147 159]
  [123 147 159]
  [124 148 160]]] in 109 length datastore
24-05-15 10:34:29 - find function duration 0.43795013427734375 seconds
1/1 [==============================] - 0s 22ms/step
24-05-15 10:34:30 - Searching [[[178 155 135]
  [178 155 135]
  [178 1

24-05-15 10:34:35 - find function duration 0.46756863594055176 seconds
1/1 [==============================] - 0s 23ms/step
24-05-15 10:34:35 - Searching [[[156 138 117]
  [159 139 118]
  [162 139 118]
  ...
  [125 141 142]
  [126 139 140]
  [126 139 140]]

 [[152 136 115]
  [155 136 115]
  [159 137 116]
  ...
  [128 139 140]
  [128 139 140]
  [128 139 140]]

 [[151 134 115]
  [152 135 115]
  [154 136 115]
  ...
  [126 140 138]
  [126 140 139]
  [125 139 137]]

 ...

 [[130 122 100]
  [132 121 101]
  [135 121 101]
  ...
  [107 138 140]
  [109 138 140]
  [108 137 139]]

 [[129 120 100]
  [131 121 101]
  [132 121 101]
  ...
  [106 136 141]
  [111 137 142]
  [110 136 140]]

 [[129 119 102]
  [132 120 103]
  [135 120 103]
  ...
  [110 135 143]
  [112 135 142]
  [112 135 142]]] in 109 length datastore
24-05-15 10:34:35 - find function duration 0.4687783718109131 seconds
1/1 [==============================] - 0s 21ms/step
24-05-15 10:34:35 - Searching [[[161 140 119]
  [159 141 119]
  [155 14

24-05-15 10:34:41 - find function duration 0.53409743309021 seconds
1/1 [==============================] - 0s 21ms/step
24-05-15 10:34:41 - Searching [[[160 138 115]
  [160 138 116]
  [160 138 117]
  ...
  [132 139 140]
  [132 139 140]
  [132 139 140]]

 [[158 136 113]
  [157 136 114]
  [156 136 115]
  ...
  [131 139 138]
  [132 139 139]
  [131 138 137]]

 [[154 134 111]
  [152 134 112]
  [149 134 112]
  ...
  [132 139 139]
  [133 138 137]
  [132 137 136]]

 ...

 [[132 121 100]
  [135 121 100]
  [137 121 100]
  ...
  [115 135 136]
  [115 135 136]
  [116 136 137]]

 [[135 121 100]
  [136 121  99]
  [137 122  98]
  ...
  [116 137 135]
  [116 137 135]
  [116 137 135]]

 [[135 121 100]
  [136 121  99]
  [137 122  98]
  ...
  [117 135 136]
  [117 135 136]
  [118 136 137]]] in 109 length datastore
24-05-15 10:34:41 - find function duration 0.4404151439666748 seconds
1/1 [==============================] - 0s 28ms/step
24-05-15 10:34:41 - Searching [[[155 140 118]
  [155 140 118]
  [156 141 1

24-05-15 10:34:46 - find function duration 0.4390075206756592 seconds
1/1 [==============================] - 0s 21ms/step
24-05-15 10:34:47 - Searching [[[162 142 121]
  [160 142 121]
  [158 142 121]
  ...
  [131 140 146]
  [130 140 146]
  [130 140 146]]

 [[157 139 118]
  [156 140 118]
  [155 140 118]
  ...
  [130 141 144]
  [130 141 144]
  [130 141 144]]

 [[153 137 116]
  [153 137 117]
  [153 137 118]
  ...
  [126 138 144]
  [128 138 144]
  [128 138 144]]

 ...

 [[138 122 101]
  [137 124 102]
  [136 125 103]
  ...
  [114 137 147]
  [114 137 148]
  [114 136 148]]

 [[139 123 104]
  [138 123 103]
  [138 125 103]
  ...
  [111 136 144]
  [113 136 145]
  [114 137 146]]

 [[139 123 104]
  [138 123 103]
  [138 125 103]
  ...
  [111 136 141]
  [113 137 142]
  [113 137 141]]] in 109 length datastore
24-05-15 10:34:47 - find function duration 0.42386674880981445 seconds
1/1 [==============================] - 0s 20ms/step
24-05-15 10:34:47 - Searching [[[164 141 121]
  [164 141 121]
  [165 14

...

24-05-15 10:34:59 - find function duration 0.43193793296813965 seconds
1/1 [==============================] - 0s 20ms/step
24-05-15 10:34:59 - Searching [[[164 143 127]
  [165 144 128]
  [165 144 128]
  ...
  [127 146 151]
  [127 146 151]
  [126 146 151]]

 [[160 141 122]
  [160 142 121]
  [161 143 122]
  ...
  [127 146 149]
  [125 145 148]
  [125 145 148]]

 [[156 140 121]
  [158 141 122]
  [158 141 122]
  ...
  [131 144 150]
  [127 144 150]
  [126 143 149]]

 ...

 [[135 126 104]
  [135 126 104]
  [135 126 104]
  ...
  [119 145 150]
  [119 145 150]
  [119 145 150]]

 [[135 126 104]
  [135 126 105]
  [135 126 106]
  ...
  [120 144 149]
  [120 144 149]
  [119 143 147]]

 [[134 125 105]
  [132 125 106]
  [131 126 108]
  ...
  [114 141 149]
  [118 141 148]
  [118 141 148]]] in 109 length datastore
24-05-15 10:34:59 - find function duration 0.40262794494628906 seconds
1/1 [==============================] - 0s 20ms/step
24-05-15 10:35:00 - Searching [[[164 144 123]
  [163 144 123]
  [162 1

24-05-15 10:35:04 - find function duration 0.423814058303833 seconds
1/1 [==============================] - 0s 21ms/step
24-05-15 10:35:05 - Searching [[[165 148 126]
  [169 149 128]
  [171 148 128]
  ...
  [131 147 152]
  [132 147 152]
  [132 147 152]]

 [[163 145 126]
  [166 146 126]
  [168 147 126]
  ...
  [128 148 150]
  [128 148 150]
  [128 148 150]]

 [[159 143 124]
  [160 144 124]
  [161 145 124]
  ...
  [131 147 151]
  [131 146 149]
  [131 146 149]]

 ...

 [[136 127 107]
  [136 127 107]
  [136 127 107]
  ...
  [114 143 154]
  [116 143 154]
  [115 142 153]]

 [[135 126 104]
  [135 126 104]
  [135 126 104]
  ...
  [111 143 154]
  [112 143 154]
  [112 143 154]]

 [[135 126 104]
  [135 126 104]
  [135 126 104]
  ...
  [112 143 154]
  [114 143 154]
  [114 143 154]]] in 109 length datastore
24-05-15 10:35:05 - find function duration 0.4433131217956543 seconds
1/1 [==============================] - 0s 19ms/step
24-05-15 10:35:05 - Searching [[[169 148 129]
  [169 148 128]
  [169 149 

24-05-15 10:35:10 - find function duration 0.4639897346496582 seconds
1/1 [==============================] - 0s 21ms/step
24-05-15 10:35:10 - Searching [[[165 149 128]
  [166 150 130]
  [166 150 131]
  ...
  [132 148 153]
  [133 148 153]
  [133 148 153]]

 [[162 146 127]
  [161 146 127]
  [161 147 128]
  ...
  [131 149 151]
  [133 148 151]
  [132 147 150]]

 [[159 143 124]
  [160 144 125]
  [161 145 126]
  ...
  [133 147 150]
  [134 147 148]
  [133 146 147]]

 ...

 [[136 127 105]
  [137 129 107]
  [137 129 107]
  ...
  [118 144 154]
  [119 144 153]
  [119 144 153]]

 [[137 128 108]
  [137 128 108]
  [136 127 105]
  ...
  [120 143 153]
  [121 144 153]
  [120 143 152]]

 [[136 126 109]
  [136 126 109]
  [136 126 109]
  ...
  [120 143 153]
  [120 143 150]
  [121 145 151]]] in 109 length datastore
24-05-15 10:35:11 - find function duration 0.4383513927459717 seconds
1/1 [==============================] - 0s 20ms/step
24-05-15 10:35:11 - Searching [[[165 149 128]
  [166 150 129]
  [165 149

24-05-15 10:35:16 - find function duration 0.6074619293212891 seconds
1/1 [==============================] - 0s 20ms/step
24-05-15 10:35:16 - Searching [[[165 149 128]
  [166 150 129]
  [166 150 129]
  ...
  [133 148 153]
  [132 147 152]
  [132 147 152]]

 [[162 147 125]
  [163 148 126]
  [163 148 126]
  ...
  [132 147 150]
  [136 147 150]
  [137 147 150]]

 [[160 144 123]
  [160 144 124]
  [161 145 126]
  ...
  [131 147 151]
  [134 146 152]
  [133 145 151]]

 ...

 [[139 126 104]
  [139 126 104]
  [140 127 105]
  ...
  [115 144 155]
  [117 144 155]
  [116 143 154]]

 [[138 127 107]
  [138 127 106]
  [139 128 107]
  ...
  [114 143 154]
  [114 143 154]
  [114 143 154]]

 [[135 125 108]
  [135 126 106]
  [136 127 105]
  ...
  [115 143 152]
  [114 143 154]
  [114 143 154]]] in 109 length datastore
24-05-15 10:35:16 - find function duration 0.4254608154296875 seconds
1/1 [==============================] - 0s 20ms/step
24-05-15 10:35:17 - Searching [[[170 147 126]
  [169 147 126]
  [168 147

24-05-15 10:35:21 - find function duration 0.44147729873657227 seconds
1/1 [==============================] - 0s 21ms/step
24-05-15 10:35:22 - Searching [[[162 147 125]
  [163 148 126]
  [163 148 126]
  ...
  [134 148 153]
  [135 148 153]
  [135 148 153]]

 [[160 144 123]
  [161 145 124]
  [161 145 124]
  ...
  [132 148 152]
  [130 147 150]
  [130 147 150]]

 [[157 143 122]
  [158 143 122]
  [158 142 121]
  ...
  [130 148 153]
  [128 147 152]
  [126 146 151]]

 ...

 [[137 129 107]
  [137 129 107]
  [137 129 107]
  ...
  [118 144 154]
  [119 144 153]
  [120 145 154]]

 [[137 129 107]
  [137 129 107]
  [137 129 107]
  ...
  [121 144 154]
  [121 145 152]
  [122 146 152]]

 [[136 127 105]
  [137 129 107]
  [137 129 107]
  ...
  [120 144 148]
  [121 145 150]
  [121 145 150]]] in 109 length datastore
24-05-15 10:35:22 - find function duration 0.46193599700927734 seconds
1/1 [==============================] - 0s 20ms/step
24-05-15 10:35:22 - Searching [[[168 152 131]
  [170 152 131]
  [172 1